In [43]:
### written by Scott McCain, June 2019
### Code audit done: June 26, 2019.

import pandas as pd
from pandas import DataFrame
import numpy as np
import csv
import os

from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import random


In [74]:
# read in the annotation tab files

def read_in_annot(file_in):
    '''
    # function to read in the annotation file (.tab) associated with the sequence file
    '''
    # make a list of annotation lines
    annot_list = []
    # open file
    with open(file_in,
             newline = '') as meta_annot:
        annot_reader = csv.reader(meta_annot, delimiter='\t')
        for meta_annot in annot_reader:
            # add each annotation line to the list
            annot_list.append(meta_annot) 
    # convert the list into a pd dataframe
    annot_df = DataFrame.from_records(annot_list[1:], columns = annot_list[0])
    # format the columns of numbers correctly:
    return annot_df

# tester = read_in_annot(file_in = '../data/metaGT/AAHZ_L5_ANT_pool1_MetaG/metag_annotation_all.filtered.grps.LPI.RPKMs.GO.tab')

def read_and_format(file_in, column_low, column_high):
    '''
    reads in using the above function, and converts columns to numeric
    '''
    if 'tab' in file_in:
        unformat_df = read_in_annot(file_in = file_in)
    if 'xlsx' in file_in:
        unformat_df = pd.read_excel(file_in)
    cols_to_convert = unformat_df.columns[column_low:column_high]
    print('Converting the following columns to numeric:', cols_to_convert)
    unformat_df[cols_to_convert] = unformat_df[cols_to_convert].apply(pd.to_numeric, 
                                                               errors = 'coerce')
    # for each col to convert, go in an ensure that the minimum is greater than zero
    for i in range(len(cols_to_convert)):
        col_min = unformat_df[cols_to_convert[i]].min()
        if col_min < 0:
            raise ValueError('Are you sure you chose the right columns? Seems that they dont go 0 to 1')
    return unformat_df


metag_df = read_and_format(file_in = '../data/metaGT/AAHZ_L5_ANT_pool1_MetaG/metag_annotation_all.filtered.grps.LPI.RPKMs.GO.tab',
                          column_low = 35, column_high = 47)
metat_df = read_and_format(file_in = '../data/metaGT/AAHZ_L6_ANT_pool2_MetaT/metat_annotation_all.filtered.grps.LPI.RPKMs.GO.tab',
                          column_low = 35, column_high = 53)
tfg_df = read_and_format(file_in = '../data/mcmurdo-metatrans/Bertrand_McCrow_TFG/annotation_allTFG.grpnorm_mmetsp_fc_pn_reclassified.edgeR.xlsx',
                        column_low = 46, column_high = 49)

# # read in meta G
# metag_list = []
# with open('../data/metaGT/AAHZ_L5_ANT_pool1_MetaG/metag_annotation_all.filtered.grps.LPI.RPKMs.GO.tab',
#          newline = '') as metag_annot:
#     metag_annot_reader = csv.reader(metag_annot, delimiter='\t')
#     for metag_annot in metag_annot_reader:
#         metag_list.append(metag_annot)
# # convert the list into a pd dataframe
# metag_df = DataFrame.from_records(metag_list)
# tester = metag_list[1:]
# tester2 = DataFrame.from_records(tester, columns=metag_list[0])

# # read in meta T
# metat_list = []
# with open('../data/metaGT/AAHZ_L6_ANT_pool2_MetaT/metat_annotation_all.filtered.grps.LPI.RPKMs.GO.tab',
#          newline = '') as metat_annot:
#     metat_annot_reader = csv.reader(metat_annot, delimiter='\t')
#     for metat_annot in metat_annot_reader:
#         metat_list.append(metat_annot)
# # convert the list into a pd dataframe
# metat_df = DataFrame.from_records(metat_annot)

Converting the following columns to numeric: Index(['GOS_927_0_1G_S63_L005_001', 'GOS_927_0_8G_S62_L005_001',
       'GOS_927_3_0G_S61_L005_001', 'GOS_930_0_1G_S66_L005_001',
       'GOS_930_0_8G_S65_L005_001', 'GOS_930_3_0G_S64_L005_001',
       'GOS_933_0_1G_S69_L005_001', 'GOS_933_0_8G_S68_L005_001',
       'GOS_933_3_0G_S67_L005_001', 'GOS_935_0_1G_S72_L005_001',
       'GOS_935_0_8G_S71_L005_001', 'GOS_935_3_0G_S70_L005_001'],
      dtype='object')
Converting the following columns to numeric: Index(['GOS_927_0_1T_S75_L006_001', 'GOS_927_0_8T_S74_L006_001',
       'GOS_927_3_0T_S73_L006_001', 'GOS_929_0_1T_S78_L006_001',
       'GOS_929_0_8T_S77_L006_001', 'GOS_929_3_0T_S76_L006_001',
       'GOS_930_0_1T_S81_L006_001', 'GOS_930_0_8T_S80_L006_001',
       'GOS_930_3_0T_S79_L006_001', 'GOS_932_0_1T_S84_L006_001',
       'GOS_932_0_8T_S83_L006_001', 'GOS_932_3_0T_S82_L006_001',
       'GOS_933_0_1T_S87_L006_001', 'GOS_933_0_8T_S86_L006_001',
       'GOS_933_3_0T_S85_L006_001', 'GOS_9

In [3]:
# read in the all assembly files
def read_in_seqs(file_in):
    seq_vec = []
    orf_vec = []
    for seq_record in SeqIO.parse(file_in, 'fasta'):
        seq_vec.append(seq_record.seq)
        orf_vec.append(seq_record.id)
    if len(seq_vec) != len(orf_vec):
        raise ValueError('Something seems odd? The ORF vector length is greater than the sequence vector.')
    # making a dictionary of values, where the keys are ORF ID's and the sequences are the values
    return_dict = dict(zip(orf_vec, seq_vec))
    return return_dict

# read in meta G faa
metag_seqs = read_in_seqs(file_in = '../data/metaGT/AAHZ_L5_ANT_pool1_MetaG/assembly_all/metag_assembly.orf.faa')

# read in meta T faa
metat_seqs = read_in_seqs(file_in = '../data/metaGT/AAHZ_L6_ANT_pool2_MetaT/assembly_all/metat_assembly.orf.faa')

# read in TFG metaT
tfg_seqs = read_in_seqs(file_in = '../data/mcmurdo-metatrans/assembly.orf.fasta')

In [64]:
metat_df.describe()

,GOS_927_0_1T_S75_L006_001,GOS_927_0_8T_S74_L006_001,GOS_927_3_0T_S73_L006_001,GOS_929_0_1T_S78_L006_001,GOS_929_0_8T_S77_L006_001,GOS_929_3_0T_S76_L006_001,GOS_930_0_1T_S81_L006_001,GOS_930_0_8T_S80_L006_001,GOS_930_3_0T_S79_L006_001,GOS_932_0_1T_S84_L006_001,GOS_932_0_8T_S83_L006_001,GOS_932_3_0T_S82_L006_001,GOS_933_0_1T_S87_L006_001,GOS_933_0_8T_S86_L006_001,GOS_933_3_0T_S85_L006_001,GOS_935_0_1T_S90_L006_001,GOS_935_0_8T_S89_L006_001,GOS_935_3_0T_S88_L006_001
count,87162.000000,87162.000000,87162.000000,87162.000000,87162.000000,87162.000000,87162.000000,87162.000000,87162.000000,87162.00000,87162.000000,87162.000000,87162.000000,87162.000000,87162.000000,87162.000000,87162.00000,87162.000000
mean,15.531291,13.954075,8.309466,14.543993,13.092815,6.829812,14.925158,14.238469,13.112753,14.33845,14.851799,13.712626,14.803767,13.969085,11.312234,14.798458,13.67389,8.552823
std,190.118185,115.116746,96.970095,172.648194,109.133300,59.804126,120.159475,84.136698,118.361058,77.20987,73.503325,81.464777,145.112303,83.338751,233.035375,136.171317,67.22852,141.869407
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.450000,0.000000,0.000000,0.000000,0.000000,0.00000,0.630000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.250000,0.780000,0.000000,1.600000,2.120000,1.120000,1.220000,1.460000,0.000000,2.26000,3.020000,1.930000,1.860000,1.170000,0.000000,1.700000,2.75000,0.000000
75%,5.670000,6.240000,0.000000,8.260000,6.860000,4.970000,6.370000,7.560000,4.630000,10.02000,9.200000,7.550000,7.100000,7.670000,0.000000,7.780000,10.12000,0.000000
max,36569.630000,10892.760000,10794.510000,34017.800000,17978.060000,14434.730000,11555.710000,7638.750000,21520.100000,9916.01000,4440.270000,5472.080000,13659.180000,6189.630000,53429.230000,17011.550000,11836.01000,33728.070000


In [73]:
# tester = get_sample_specific_orfs(column_to_select1 = 'GOS_927_0_1G_S63_L005_001', df1 = metag_df,
#                        column_to_select2 = 'GOS_927_0_1T_S75_L006_001', df2 = metat_df)

type(metag_df['GOS_927_0_1G_S63_L005_001'][0])

str

In [103]:
# subset the orfs based on the column completeness for a given sample ID
         
# def get_sample_specific_orfs(column_to_select1, df1, 
#                              column_to_select2, df2):
    
# #     column_to_select1 = 'GOS_927_0_1G_S63_L005_001'
#     # boolean which of this column is greater than 0
#     # return the orf_id values from that
#     print('Finding ORFs present...')
#     orfs_present1 = df1[df1[column_to_select1] > 0]['orf_id'].tolist()
#     orfs_present2 = df2[df2[column_to_select2] > 0]['orf_id'].tolist()
#     return [orfs_present1, orfs_present2]


def get_sample_specific_orfs(column_to_select1, df1, 
                             column_to_select2, df2):
    
    ## this function goes into the master df and returns orfs which were present in a given sample
    
    print('Finding ORFs present...')
    
    if df2 is None or column_to_select2 is None:
        ## selected only one dataframe to subsample from
        orfs_present1_redundant = []
        for column_i in range(len(column_to_select1)):

            orfs_present1_temp = df1[df1[column_to_select1[column_i]] > 0]['orf_id'].tolist()
            orfs_present1_redundant = orfs_present1_temp + orfs_present1_redundant
        
        orfs_present1 = list(set(orfs_present1_redundant))
        
        return [orfs_present1]

    else:
        
        if isinstance(column_to_select1, str):
            print('One column for each df selected...')
            orfs_present1 = df1[df1[column_to_select1] > 0]['orf_id'].tolist()
            orfs_present2 = df2[df2[column_to_select2] > 0]['orf_id'].tolist()

        if isinstance(column_to_select1, list):
            print('Multiple columns for each df selected...')

            orfs_present1_redundant = []
            for column_i in range(len(column_to_select1)):

                orfs_present1_temp = df1[df1[column_to_select1[column_i]] > 0]['orf_id'].tolist()
                orfs_present1_redundant = orfs_present1_temp + orfs_present1_redundant

            orfs_present2_redundant = []
            for column_i in range(len(column_to_select2)):
                orfs_present2_temp = df2[df2[column_to_select2[column_i]] > 0]['orf_id'].tolist()
                orfs_present2_redundant = orfs_present2_temp + orfs_present2_redundant

            orfs_present1 = list(set(orfs_present1_redundant))
            orfs_present2 = list(set(orfs_present2_redundant))

        return [orfs_present1, orfs_present2]

# tester = get_sample_specific_orfs(column_to_select1 = ['TFG_t0_A', 'TFG_t0_B', 'TFG_t0_C'], 
#                                   df1 = tfg_df, 
#                                   column_to_select2 = None, df2 = None)
    
# tester = get_sample_specific_orfs(column_to_select1 = 'GOS_927_0_1G_S63_L005_001', df1 = metag_df,
#                        column_to_select2 = 'GOS_927_0_1T_S75_L006_001', df2 = metat_df)


# tester = get_sample_specific_orfs(column_to_select1 = ['GOS_927_0_1G_S63_L005_001', 
#                                                        'GOS_935_0_1G_S72_L005_001'], df1 = metag_df,
#                         column_to_select2 = ['GOS_927_0_1T_S75_L006_001',
#                                              'GOS_935_0_1T_S90_L006_001'], df2 = metat_df)

def merge_dict(dict1, dict2):
    return dict2.update(dict1)

# function to subsample the master sample and return a list of dictionaries that contain the new data
# def subsample_master_assembly(orfs_present_lists, assembly1, assembly2):
    
# #     assembly1 = metat_seqs
# #     assembly2 = metag_seqs
# #     orfs_present_list1 = tester[0]
# #     orfs_present_list2 = tester[1]
#     orfs_present_list1 = orfs_present_lists[0]
#     orfs_present_list2 = orfs_present_lists[1]
    
#     copy_dict1 = dict(assembly1)
#     for (key, value) in copy_dict1.items():
#         if key not in orfs_present_list1:
#             del assembly1[key]

#     copy_dict2 = dict(assembly2)
#     for (key, value) in copy_dict2.items():
#         if key not in orfs_present_list2:
#             del assembly2[key]
    
#     merged_dicts = merge_dict(dict1=assembly1, dict2=assembly2)

#     return [assembly1, assembly2]

# test_subsample = subsample_master_assembly(orfs_present_lists=tester,
#                                           assembly1=metag_seqs,
#                                           assembly2=metat_seqs)
# the above function seems to work, but it takes a while. instead of subsampling the master,
# by looping through each entry to see if it's in the orfs_present_list, it might be faster
# to go through the orfs_present_list and grab the elements of the dictionary that correspond to these
# orfs, and then output that.

def collect_from_master_assembly(orfs_present_lists, assembly_list):
    print('Collecting ORFs from master assembly, this step takes time...')
#     assembly_list = [assembly1, assembly2]

    subsample_sequences = []
    subsample_orf_ids = []
    
    # go through each ORF ID list 
    print(len(orfs_present_lists))
    for orf_present_list_i in range(len(orfs_present_lists)):
        # get the ith ORF ID list and the ith Assembly file
        single_orf_list = orfs_present_lists[orf_present_list_i]
        assembly_i = assembly_list[orf_present_list_i]
        
        # for each ORF, file it in the assembly file, and ad it to the subsample sequence file
        for orf_i in range(len(single_orf_list)):
            # find the sequence based on the key of the dictionary of the assembly
            target_seq = assembly_i[single_orf_list[orf_i]]
            subsample_sequences.append(target_seq)
            subsample_orf_ids.append(single_orf_list[orf_i])
            
        if len(subsample_sequences) != len(subsample_orf_ids):
            raise ValueError('A very specific bad thing happened, not sure why..')
    
    return [subsample_sequences, subsample_orf_ids]
            


# function to take these dictionaries and write one fasta file with them
def write_fasta_file(subsample_output, fasta_name_in):
    
    fasta_name = str(fasta_name_in) + '.fasta'
    
    if fasta_name[-5:] != 'fasta':
        raise NameError('file input name does not have proper ending')
    
    print('Writing fasta file...')
    with open(fasta_name, 'w') as ofile:
        for i in range(len(subsample_output[0])):
            ofile.write(">" + str(subsample_output[1][i]) + "\n" + str(subsample_output[0][i]) + "\n")
    ofile.close()


def check_list_of_names(name_list):
    print(name_list)
    sub_flat_list = []
    for i in range(len(name_list)):
        sub_flat_list.append(name_list[i][8:11])
    unique_elements = list(set(sub_flat_list))
    print(unique_elements)
    if len(unique_elements) > 1:
        raise NameError('houston, you have a problem. Are you sure the input files are all from the same size fraction?')
        
        
def check_names(name_1, name_2):
    
    if isinstance(name_1, str):
        name_1_sub = name_1[0:11]
        name_2_sub = name_2[0:11]
    
    if isinstance(name_1, list):
        # check that the samples are all of the sample size class
#         check_list_of_names(name_2)
#         check_list_of_names(name_1)
        
        # make a return name
        name_1_sub = 'GOS_' + str(name_1[0][8:11])
        name_2_sub = 'GOS_' + str(name_2[0][8:11])

    print('saving to file name:', name_1_sub)
    if name_1_sub != name_2_sub:
        raise NameError('Input files for transcriptomic does not match genomic, eek!')
    
    return name_1_sub

def write_multi_fastas(list_of_column_names, df1, df2, assembly_list):
    # loop through each element of the list, which is also a 2-element list of the two columns
    # that are subsampled
    for pair_of_columns in range(len(list_of_column_names)):
        
        column_name_1 = list_of_column_names[pair_of_columns][0]
        column_name_2 = list_of_column_names[pair_of_columns][1]
        
        file_name_to_save = check_names(name_1 = column_name_1,
                                       name_2 = column_name_2)
        
        sample_orf_lists = get_sample_specific_orfs(column_to_select1 = column_name_1, 
                                                    df1 = df1, 
                                                    column_to_select2 = column_name_2, 
                                                    df2 = df2)
        subsampled_for_fasta = collect_from_master_assembly(orfs_present_lists = sample_orf_lists, 
                                                            assembly_list = assembly_list)
        write_fasta_file(subsample_output = subsampled_for_fasta, 
                         fasta_name_in = file_name_to_save)
        
def write_one_fasta(list_of_column_names, df1, assembly_list, custom_name):
    found_orfs = get_sample_specific_orfs(column_to_select1 = list_of_column_names, 
                                      df1 = df1, 
                                      column_to_select2 = None, 
                                      df2 = None)
    subsample_orfs = collect_from_master_assembly(orfs_present_lists=found_orfs,
                                              assembly_list=assembly_list)
    write_fasta_file(subsample_orfs, fasta_name_in = custom_name)
    

In [104]:
    
# list of columns (sample IDs) that pair up the metagenomic and metatranscriptomic sequences
# for creating fasta files
column_pair_list = [['GOS_927_0_1G_S63_L005_001', 'GOS_927_0_1T_S75_L006_001'], 
                     ['GOS_927_0_8G_S62_L005_001', 'GOS_927_0_8T_S74_L006_001'], 
                     ['GOS_927_3_0G_S61_L005_001', 'GOS_927_3_0T_S73_L006_001'], 
                     ['GOS_930_0_1G_S66_L005_001', 'GOS_930_0_1T_S81_L006_001'],
                     ['GOS_930_0_8G_S65_L005_001', 'GOS_930_0_8T_S80_L006_001'],
                     ['GOS_930_3_0G_S64_L005_001', 'GOS_930_3_0T_S79_L006_001'],
                     ['GOS_933_0_1G_S69_L005_001', 'GOS_933_0_1T_S87_L006_001'],
                     ['GOS_933_0_8G_S68_L005_001', 'GOS_933_0_8T_S86_L006_001'],
                     ['GOS_933_3_0G_S67_L005_001', 'GOS_933_3_0T_S85_L006_001'],
                     ['GOS_935_0_1G_S72_L005_001', 'GOS_935_0_1T_S90_L006_001'],
                     ['GOS_935_0_8G_S71_L005_001', 'GOS_935_0_8T_S89_L006_001'],
                     ['GOS_935_3_0G_S70_L005_001', 'GOS_935_3_0T_S88_L006_001']]    

# make the column pair list that has pooled samples
column_pair_list_pooled = [[['GOS_927_0_1G_S63_L005_001','GOS_930_0_1G_S66_L005_001',
                             'GOS_933_0_1G_S69_L005_001','GOS_935_0_1G_S72_L005_001'],
                             ['GOS_927_0_1T_S75_L006_001', 'GOS_930_0_1T_S81_L006_001',
                             'GOS_933_0_1T_S87_L006_001', 'GOS_935_0_1T_S90_L006_001']],
                           [['GOS_927_0_8G_S62_L005_001', 'GOS_930_0_8G_S65_L005_001',
                            'GOS_933_0_8G_S68_L005_001', 'GOS_935_0_8G_S71_L005_001'],
                           ['GOS_927_0_8T_S74_L006_001', 'GOS_930_0_8T_S80_L006_001',
                           'GOS_933_0_8T_S86_L006_001', 'GOS_935_0_8T_S89_L006_001']],
                           [['GOS_927_3_0G_S61_L005_001', 'GOS_930_3_0G_S64_L005_001', 
                            'GOS_933_3_0G_S67_L005_001', 'GOS_935_3_0G_S70_L005_001'],
                           ['GOS_927_3_0T_S73_L006_001', 'GOS_930_3_0T_S79_L006_001', 
                           'GOS_933_3_0T_S85_L006_001', 'GOS_935_3_0T_S88_L006_001']]]

write_multi_fastas(list_of_column_names = column_pair_list, 
                  df1 = metag_df, 
                  df2 = metat_df, 
                  assembly1 = metag_seqs,
                  assembly2 = metat_seqs)

write_multi_fastas(list_of_column_names = column_pair_list_pooled, 
                  df1 = metag_df, 
                  df2 = metat_df, 
                  assembly_list = [metag_seqs, metat_seqs])

saving to file name: GOS_0_1
Finding ORFs present...
Multiple columns for each df selected...
2
Writing fasta file...
saving to file name: GOS_0_8
Finding ORFs present...
Multiple columns for each df selected...
2
Writing fasta file...
saving to file name: GOS_3_0
Finding ORFs present...
Multiple columns for each df selected...
2


KeyboardInterrupt: 

In [105]:

# make the column pair list that has pooled samples
column_tfg_list = ['TFG_t0_A', 'TFG_t0_B', 'TFG_t0_C']
# fetching from the same dataframe, since it's just metaT and I don't want to make the above code more flexible
write_one_fasta(list_of_column_names = column_tfg_list, 
                  df1 = tfg_df, assembly_list = [tfg_seqs], custom_name = "tfg_t0")


Finding ORFs present...
1
Writing fasta file...


In [44]:
# the naming convention was built for the sample specific databases
# manually rename this one:
os.rename('GOS_.fasta', 'tfg_t0.fasta')

In [49]:
tester = tfg_df[tfg_df['TFG_t0_A'] > 0]['orf_id'].tolist()


318965